In [ ]:
from experta import *
import pandas as pd

# Define your Fact class
class Course(Fact):
    pass

class User(Fact):
    pass 

class GraduationRule(Fact):
    pass

class Elective_Courses_Passed(Fact):
    pass

class Elective_University_Passed(Fact):
    pass

class University_Requirement_Passed(Fact):
    pass
class WarnningMessage(Fact):
    pass
class calculate_cgpa(Fact):
    pass

file_path = 'High Performance Computing.xlsx'

# Read the Excel file with two specified sheet names or indices
sheets = ['Sheet1', 'Sheet2', 'Sheet3']
dfs = pd.read_excel(file_path, sheet_name=sheets)

# Access the data frames for each sheet
sheet1_df = dfs['Sheet1'] # study plan
sheet2_df = dfs['Sheet2'] #elective courses
sheet3_df = dfs['Sheet3'] #university requirements and elective unvirsity

class CourseListing(KnowledgeEngine):
    @Rule(User(semesters=W(), CGPA=P(lambda x: x < 2)))
    def advise_courses_cgpa_half_load(self):
        passed_courses = input("Enter the passed courses (comma separated): ").upper().split(',')
        failed_courses = input("Enter the failed courses (comma separated): ").upper().split(',')
        advised_courses = []
        elective_codes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6']
        UniRequirement_codes = ['University Requirement']
        UniElective_codes = ['Elective University']
        total_CH = 13

        for index, row in sheet1_df.iterrows():
            if row['code'] in passed_courses:
                continue
            elif row['code'] in failed_courses and  total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']
                print(f"You failed in course {row['code']}, you should retake it in the first.")
            elif pd.notnull(row['prerequisite']) and row['prerequisite'] in failed_courses:
                continue  # Skip if the prerequisite is in the failed courses
            elif  total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']  # Print the whole row if not in passed or failed courses

        print("You are eligible to take the following courses:")
        for course in advised_courses:
            print(f"semester: {course['semester']}, Code: {course['code']}, Name: {course['course_name']}, CH: {course['course_CH']}, Type: {course['course_type']}, Prerequisites: {course['prerequisite']}")
            
            
                
#**************************************ELECTIVE COURSES*************************        
        
        # Initialize a dictionary to store whether each type has been advised
        advised_types = {code: False for code in elective_codes}
        elective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['code'] in elective_codes and not advised_types[course['code']]:
                elective_courses = sheet2_df[sheet2_df['course_type'] == course['code']]
                elective_courses_dict[course['code']] = elective_courses
                advised_types[course['code']] = True

        # Prompt the user to choose from each type
        if len(elective_courses_dict) > 0:# Check if there are any ELECTIVE courses
            for code, elective_courses in elective_courses_dict.items():
                print(f"Choose one course from Type {code}:")
                for index, row in elective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                    print("YOU CAN ONLY TAKE ONE OF EACH TYPE: ")
                chosen_course_code = input(f"Enter the code of the course you choose from Type {code}: ")
                print("YOUR CHOICE IS: ", {chosen_course_code})
                # You can add code here to handle the chosen course code
        else:
            print("No elective courses available.")    
        
        
        
        
        
        
 #****************************CORE UNIVERSITY REQUIREMENTES COURSES********************       
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniRequirement_codes}
        UniRequirement_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniRequirement_codes and not advised_types[course['course_type']]:
                UniRequirement_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniRequirement_courses_dict[course['course_type']] = UniRequirement_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniRequirement_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniRequirement_courses in UniRequirement_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniRequirement_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No University Requirement courses available.")

        
        
   #****************************ELECTIVE UNIVERSITY REQUIREMENTES COURSES********************             
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniElective_codes}
        UniElective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniElective_codes and not advised_types[course['course_type']]:
                UniElective_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniElective_courses_dict[course['course_type']] = UniElective_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniElective_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniElective_courses in UniElective_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniElective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No Elective University courses available.")

            
            
            
    @Rule(User(semesters=W(), CGPA=P(lambda x: 3 > x >= 2)))
    def advise_courses_cgpa_full_load(self):
        passed_courses = input("Enter the passed courses (comma separated): ").upper().split(',')
        failed_courses = input("Enter the failed courses (comma separated): ").upper().split(',')
        advised_courses = []
        elective_codes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6']
        UniRequirement_codes = ['University Requirement']
        UniElective_codes = ['Elective University']
        total_CH = 20

        for index, row in sheet1_df.iterrows():
            if row['code'] in passed_courses:
                continue
            elif row['code'] in failed_courses and  total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']
                print(f"You failed in course {row['code']}, you should retake it.")
            elif pd.notnull(row['prerequisite']) and row['prerequisite'] in failed_courses:
                continue  # Skip if the prerequisite is in the failed courses
            elif  total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']  # Print the whole row if not in passed or failed courses

        print("You are eligible to take the following courses:")
        for course in advised_courses:
            print(f"semester: {course['semester']}, Code: {course['code']}, Name: {course['course_name']}, CH: {course['course_CH']}, Type: {course['course_type']}, Prerequisites: {course['prerequisite']}")
            
                
#**************************************ELECTIVE COURSES*************************        
        
        # Initialize a dictionary to store whether each type has been advised
        advised_types = {code: False for code in elective_codes}
        elective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['code'] in elective_codes and not advised_types[course['code']]:
                elective_courses = sheet2_df[sheet2_df['course_type'] == course['code']]
                elective_courses_dict[course['code']] = elective_courses
                advised_types[course['code']] = True

        # Prompt the user to choose from each type
        if len(elective_courses_dict) > 0:# Check if there are any ELECTIVE courses
            for code, elective_courses in elective_courses_dict.items():
                print(f"Choose one course from Type {code}:")
                for index, row in elective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                    print("YOU CAN ONLY TAKE ONE OF EACH TYPE: ")
                chosen_course_code = input(f"Enter the code of the course you choose from Type {code}: ")
                print("YOUR CHOICE IS: ", {chosen_course_code})
                # You can add code here to handle the chosen course code
        else:
            print("No elective courses available.")    
        
        
        
        
        
        
 #****************************CORE UNIVERSITY REQUIREMENTES COURSES********************       
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniRequirement_codes}
        UniRequirement_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniRequirement_codes and not advised_types[course['course_type']]:
                UniRequirement_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniRequirement_courses_dict[course['course_type']] = UniRequirement_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniRequirement_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniRequirement_courses in UniRequirement_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniRequirement_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No University Requirement courses available.")

        
        
   #****************************ELECTIVE UNIVERSITY REQUIREMENTES COURSES********************             
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniElective_codes}
        UniElective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniElective_codes and not advised_types[course['course_type']]:
                UniElective_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniElective_courses_dict[course['course_type']] = UniElective_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniElective_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniElective_courses in UniElective_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniElective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No Elective University courses available.")

            
            
            
            
            
    @Rule(User(semesters=W(), CGPA=P(lambda x: x >= 3)))
    def advise_courses_cgpa_over_load(self):
        passed_courses = input("Enter the passed courses (comma separated): ").upper().split(',')
        failed_courses = input("Enter the failed courses (comma separated): ").upper().split(',')
        advised_courses = []
        elective_codes = ['E1', 'E2', 'E3', 'E4', 'E5', 'E6']
        UniRequirement_codes = ['University Requirement']
        UniElective_codes = ['Elective University']
        total_CH = 22

        for index, row in sheet1_df.iterrows():
            if row['code'] in passed_courses:
                continue
            elif row['code'] in failed_courses and  total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']
                print(f"You failed in course {row['code']}, you should retake it.")
            elif pd.notnull(row['prerequisite']) and row['prerequisite'] in failed_courses:
                continue  # Skip if the prerequisite is in the failed courses
            elif total_CH >= row['course_CH']:
                advised_courses.append(row)
                total_CH -= row['course_CH']  # Print the whole row if not in passed or failed courses

        print("You are eligible to take the following courses:")
        for course in advised_courses:
            print(f"semester: {course['semester']}, Code: {course['code']}, Name: {course['course_name']}, CH: {course['course_CH']}, Type: {course['course_type']}, Prerequisites: {course['prerequisite']}")
        
        
#**************************************ELECTIVE COURSES*************************        
        
        # Initialize a dictionary to store whether each type has been advised
        advised_types = {code: False for code in elective_codes}
        elective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['code'] in elective_codes and not advised_types[course['code']]:
                elective_courses = sheet2_df[sheet2_df['course_type'] == course['code']]
                elective_courses_dict[course['code']] = elective_courses
                advised_types[course['code']] = True

        # Prompt the user to choose from each type
        if len(elective_courses_dict) > 0:# Check if there are any ELECTIVE courses
            for code, elective_courses in elective_courses_dict.items():
                print(f"Choose one course from Type {code}:")
                for index, row in elective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                    print("YOU CAN ONLY TAKE ONE OF EACH TYPE: ")
                chosen_course_code = input(f"Enter the code of the course you choose from Type {code}: ")
                print("YOUR CHOICE IS: ", {chosen_course_code})
                # You can add code here to handle the chosen course code
        else:
            print("No elective courses available.")    
        
        
        
        
        
        
 #****************************CORE UNIVERSITY REQUIREMENTES COURSES********************       
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniRequirement_codes}
        UniRequirement_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniRequirement_codes and not advised_types[course['course_type']]:
                UniRequirement_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniRequirement_courses_dict[course['course_type']] = UniRequirement_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniRequirement_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniRequirement_courses in UniRequirement_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniRequirement_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No University Requirement courses available.")

        
        
   #****************************ELECTIVE UNIVERSITY REQUIREMENTES COURSES********************             
        advised_codes = [course['course_type'] for course in advised_courses]

        # Initialize a dictionary to store whether each type has been advised
        advised_types = {course_type: False for course_type in UniElective_codes}
        UniElective_courses_dict = {}

        # Return a list of courses from sheet2_df with course_type matching advised courses with codes E1-E6
        for course in advised_courses:
            if course['course_type'] in UniElective_codes and not advised_types[course['course_type']]:
                UniElective_courses = sheet3_df[sheet3_df['course_type'] == course['course_type']]
                UniElective_courses_dict[course['course_type']] = UniElective_courses
                advised_types[course['course_type']] = True

        # Prompt the user to choose from each type
        if len(UniElective_courses_dict) > 0:  # Check if there are any University Requirement courses
            for course_type, UniElective_courses in UniElective_courses_dict.items():
                print(f"Choose course from {course_type}:")
                for index, row in UniElective_courses.iterrows():
                    print(f"Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}")
                
                chosen_course_code = input(f"Enter the code of the course you choose from Type {course_type}: ")
                print("YOUR CHOICE IS: ", chosen_course_code)
                # You can add code here to handle the chosen course code
        else:
            print("No Elective University courses available.")
            

    @Rule(User(semesters='summer', CGPA=W()))
    def advise_courses_summer_semester(self):
        passed_courses = input("Enter the passed courses (comma separated): ").upper().split(',')
        failed_courses = input("Enter the failed courses (comma separated): ").upper().split(',')
        advised_courses = []
        total_CH = 9

        for index, row in sheet1_df.iterrows():
            
            if row['code'] in passed_courses:
                continue
            elif row['code'] in failed_courses:
                advised_courses.append(row)
                total_CH -= row['course_CH']
                print(f"You failed in course {row['code']}, you should retake it in the first.")
                
        

        print("You are eligible to take the following courses first :")
        for course in advised_courses:
            print(f"semester: {course['semester']}, Code: {course['code']}, Name: {course['course_name']}, CH: {course['course_CH']}, Type: {course['course_type']}, Prerequisites: {course['prerequisite']}")
            
        while True:
            
            choice = input("Do you want to choose other courses from the list? (yes/no): ")
            if choice.lower() != 'yes':
                break
                
            print("You can choose any course from the following courses only 9CH in summer semester:")
            for index, row in sheet1_df.iterrows():
                if row['code'] not in passed_courses and pd.isnull(row['prerequisite']):
                    print(f"semester: {row['semester']}, Code: {row['code']}, Name: {row['course_name']}, CH: {row['course_CH']}, Type: {row['course_type']}, Prerequisites: {row['prerequisite']}")
              
            choice1 = input("Enter your courses semester (comma separated): ").split(',')
            choice2 = input("Enter your courses code (comma separated): ").split(',')
            choice3 = input("Enter your courses course_name (comma separated): ").split(',')
            choice4 = input("Enter your sum_courses course_CH: ")
            choice5 = input("Enter your courses type (comma separated): ").split(',')

            print("You take the following courses list:")
            print(f"courses_CH: {choice4}, courses semester: {choice1}, courses code: {choice2}, courses names: {choice3}, courses type: {choice5}")
            
            total_CH -= int(choice4)
            if total_CH < 0:
                print("You can't register more than 9 CH. Please edit your choices. because now your credit hours is more than 9 you must choose : ", )
                continue

            new_courses = []
            for i in range(len(choice1)):
                new_courses.append({
                    'semester': choice1[i],
                    'code': choice2[i],
                    'course_name': choice3[i],
                    'course_CH': int(choice4),
                    'course_type': choice5[i],
                    'prerequisite': None
                })

            print("You have chosen the following courses:")
            for course in new_courses:
                print(f"semester: {course['semester']}, Code: {course['code']}, Name: {course['course_name']}, CH: {course['course_CH']}, Type: {course['course_type']}, Prerequisites: {course['prerequisite']}")

                
    @Rule(WarnningMessage(user_input=MATCH.user_input))
    def WarnningMessages(self,user_input):
        print("Be careful you have been put into the alert list and half load student.")
        
        
    
    # Rule to check if the student has achieved the required number of credit hours for graduation
    @Rule(GraduationRule(credit_hours=MATCH.credit_hours))
    def graduation_credit_hours_remaining(self, credit_hours):
        if credit_hours == 157:
            print("Congratulations! You have fulfilled the graduation requirement by achieved 157 credit hours.")
        else:
            remaining_ch = 157 - credit_hours
            print(f"You need {remaining_ch} credit hours to meet the graduation requirement of 157 credit hours.")

    @Rule(Elective_Courses_Passed(credit_hours_elective=MATCH.credit_hours_elective,
                               numOfElectiveCourse=MATCH.numOfElectiveCourse))
    def Elective_Courses_Passed(self, credit_hours_elective, numOfElectiveCourse):
        
        if credit_hours_elective == 18 and numOfElectiveCourse == 6:
            print("Congratulations! You have fulfilled one of the graduation requirements by achieved 18CH in elective courses.")
        else:
            remaining_ch = 18 - credit_hours_elective
            remaining_numOfCourses = 6 - numOfElectiveCourse
            print(f"You need {remaining_ch} credit hours to meet the graduation requirement of 18 credit hours.",
                  f"You need {remaining_numOfCourses} courses to meet the graduation requirement of 6 elective courses.")

    @Rule(Elective_University_Passed(credit_hours_elective_Uni=MATCH.credit_hours_elective_Uni,
                                  numOfElectiveUni=MATCH.numOfElectiveUni))
    def Elective_Uni_Passed(self, credit_hours_elective_Uni, numOfElectiveUni):
        
        if credit_hours_elective_Uni == 8 and numOfElectiveUni == 4:
            print("Congratulations! You have fulfilled one of the graduation requirements by achieved 8CH in elective university courses.")
        else:
            remaining_ch = 8 - credit_hours_elective_Uni
            remaining_numOfCourses = 4 - numOfElectiveUni
            print(f"You need {remaining_ch} credit hours to meet the graduation requirement of 8 credit hours.",
                  f"You need {remaining_numOfCourses} courses to meet the graduation requirement of 4 elective university courses.")

    @Rule(University_Requirement_Passed(credit_hours_Uni_Requirement=MATCH.credit_hours_Uni_Requirement,
                                     numOfUniRequirement=MATCH.numOfUniRequirement))
    def Uni_Requirement_Passed(self, credit_hours_Uni_Requirement, numOfUniRequirement):
        if credit_hours_Uni_Requirement == 12 and numOfUniRequirement == 6:
            print("Congratulations! You have fulfilled one of the graduation requirements by achieving 12CH in university requirements.")
        else:
            remaining_ch = 12 - credit_hours_Uni_Requirement
            remaining_numOfCourses = 6 - numOfUniRequirement
            print(f"You need {remaining_ch} credit hours to meet the graduation requirement of 12 credit hours.",
                  f"You need {remaining_numOfCourses} courses to meet the graduation requirement of 6 university requirements.")
    
class KnowledgeBaseEditor:
    def __init__(self, file_path):
        self.file_path = file_path
        self.dfs = pd.read_excel(file_path, sheet_name=['Sheet1', 'Sheet2', 'Sheet3'])
        self.sheet1_df = self.dfs['Sheet1']
        self.sheet2_df = self.dfs['Sheet2']
        self.sheet3_df = self.dfs['Sheet3']
        self.engine = CourseListing()
        self.engine.reset()

    def add_course(self, semester, code, name, CH, course_type, prerequisites):
        # Add the course to all DataFrames
        new_row = {'semester': semester, 'code': code, 'course_name': name, 'course_CH': CH, 'course_type': course_type, 'prerequisite': prerequisites}
        self.sheet1_df = self.sheet1_df.append(new_row, ignore_index=True)
        self.sheet2_df = self.sheet2_df.append(new_row, ignore_index=True)
        self.sheet3_df = self.sheet3_df.append(new_row, ignore_index=True)

    def edit_course(self, index, semester, code, name, CH, course_type, prerequisites):
        # Edit the course in all DataFrames
        self.sheet1_df.at[index, 'semester'] = semester
        self.sheet1_df.at[index, 'code'] = code
        self.sheet1_df.at[index, 'course_name'] = name
        self.sheet1_df.at[index, 'course_CH'] = CH
        self.sheet1_df.at[index, 'course_type'] = course_type
        self.sheet1_df.at[index, 'prerequisite'] = prerequisites

        self.sheet2_df.at[index, 'semester'] = semester
        self.sheet2_df.at[index, 'code'] = code
        self.sheet2_df.at[index, 'course_name'] = name
        self.sheet2_df.at[index, 'course_CH'] = CH
        self.sheet2_df.at[index, 'course_type'] = course_type
        self.sheet2_df.at[index, 'prerequisite'] = prerequisites

        self.sheet3_df.at[index, 'semester'] = semester
        self.sheet3_df.at[index, 'code'] = code
        self.sheet3_df.at[index, 'course_name'] = name
        self.sheet3_df.at[index, 'course_CH'] = CH
        self.sheet3_df.at[index, 'course_type'] = course_type
        self.sheet3_df.at[index, 'prerequisite'] = prerequisites

    def delete_course(self, index):
        if index in self.sheet1_df.index:
            # Delete the course from all DataFrames
            self.sheet1_df = self.sheet1_df.drop(index)
            self.sheet2_df = self.sheet2_df.drop(index)
            self.sheet3_df = self.sheet3_df.drop(index)
            print("Course deleted successfully.")
        else:
            print("Index not found in sheet1_df. No action taken.")
    def save_changes(self):
        # Save the updated DataFrames back to the Excel file
        with pd.ExcelWriter(self.file_path) as writer:
            self.sheet1_df.to_excel(writer, sheet_name='Sheet1', index=False)
            self.sheet2_df.to_excel(writer, sheet_name='Sheet2', index=False)
            self.sheet3_df.to_excel(writer, sheet_name='Sheet3', index=False)

# Usage
engine = CourseListing()
engine.reset()

# Usage
knowledge_editor = KnowledgeBaseEditor(file_path)
Email = input("enter your email address: ")
if Email.endswith("admin"):
    while True:
        action = input("Enter 'add' to add a course, 'edit' to edit a course, 'delete' to delete a course, 'download' to save changes, or 'exit' to exit: ")
        if action == 'add':
            semester = int(input("Enter semester: "))
            code = input("Enter course code: ")
            name = input("Enter course name: ")
            CH = int(input("Enter course credit hours: "))
            course_type = input("Enter course type: ")
            prerequisites = input("Enter course prerequisite: ")
            knowledge_editor.add_course(semester, code, name, CH, course_type, prerequisites)
            print("Course added successfully.")
            # Display the added course
            added_course = knowledge_editor.sheet1_df.tail(1)  # Get the last row of the DataFrame
            print("Added Course Details:")
            print(added_course)
        elif action == 'edit':
            index = int(input("Enter the index of the course to edit: "))
            semester = int(input("Enter semester: "))
            code = input("Enter course code: ")
            name = input("Enter course name: ")
            CH = int(input("Enter course credit hours: "))
            course_type = input("Enter course type: ")
            prerequisites = input("Enter course prerequisite: ")
            knowledge_editor.edit_course(index, semester, code, name, CH, course_type, prerequisites)
            print("Course edited successfully.")
        elif action == 'delete':
            index = int(input("Enter the index of the course to delete: "))
            knowledge_editor.delete_course(index)
            print("Course deleted successfully.")
        elif action == 'download':
            knowledge_editor.save_changes()
            print("Changes saved successfully.")
        elif action == 'exit':
            break

else:
    print("*******WELCOME*********")
    print("IN THE FIRST: are you get an CGPA less than 2.00 in any semester after the end of the third semester or less than 1.667 after the end of the second semester of? " )
    choice1 = input("yes or no: ")
    if choice1 == 'yes':
        user_input = choice1
        engine.declare(WarnningMessage(user_input=user_input))
        engine.run()
    else:
        print("You are in safe")


    choice = input("DO YOU WANT TO KNOW WHAT YOUR ADVISED LIST OF COURSES THAT YOU CAN TAKE IT (WRITE 'yes' OR 'no'): ")
    if choice == 'yes':
    #Usage
        engine = CourseListing()
        engine.reset()

        # Load courses from Excel file
        CGPA = float(input("Enter your CGPA: "))
        semesters = int(input("Enter your semester: "))
        engine.declare(User(CGPA=CGPA, semesters=semesters))

        # Run the engine
        engine.run()
        engine.reset


    else:
        print("thank you! see you soon ")


    choice = input("DO YOU WANT TO KNOW WHAT YOUR ACHIEVEMENT IN GRADUATION REQUIREMENTS (WRITE 'yes' OR 'no'): ")
    if choice == 'yes':
        credit_hours = int(input("Enter the sum of your CH: "))  # This can be a reference to the total credit hours the student has achieved
        engine.declare(GraduationRule(credit_hours=credit_hours))

        credit_hours_elective = int(input("Enter the sum of your elective courses CH: "))
        numOfElectiveCourse = int(input("Enter the sum number of your elective courses: "))
        engine.declare(Elective_Courses_Passed(credit_hours_elective=credit_hours_elective, numOfElectiveCourse=numOfElectiveCourse))

        credit_hours_elective_Uni = int(input("Enter your number of CH elective university: "))
        numOfElectiveUni = int(input("Enter your number of elective university: "))
        engine.declare(Elective_University_Passed(credit_hours_elective_Uni=credit_hours_elective_Uni, numOfElectiveUni=numOfElectiveUni))

        credit_hours_Uni_Requirement = int(input("Enter your number of CH university Req: "))
        numOfUniRequirement = int(input("Enter your number of CH university Req: "))
        engine.declare(University_Requirement_Passed(credit_hours_Uni_Requirement=credit_hours_Uni_Requirement, numOfUniRequirement=numOfUniRequirement))
        engine.run()
    else:
        print("Thank you!")
